In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [18]:
# Local imports
import kepler_sieve
from horizons import make_sim_horizons
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_numpy, integrate_df
from db_config import db_engine
from db_utils import sp2df, get_columns, df2db, df2csv, csv2db_stage, csv2db, csvs2db

In [3]:
from asteroid_element import get_ast_ref_elts, add_asteroid_elts, update_asteroid_elements
from asteroid_element_synch import process_dates

In [4]:
# Typing
from typing import Tuple, List

In [5]:
epoch_out = 59000
elt_dates = sp2df(sp_name='KS.GetAsteroidRefElementDates', params={'epoch':epoch_out})

In [6]:
elt_dates

,TimeID,epoch,AsteroidCount
0,60416640,41956.0,1
1,62537760,43429.0,1
2,63433440,44051.0,1
3,63678240,44221.0,1
4,64319040,44666.0,1
...,...,...,...
5242,84975840,59011.0,2
5243,84977280,59012.0,1
5244,84978720,59013.0,2
5245,84980160,59014.0,6


In [7]:
# First epoch
epoch0 = int(elt_dates.epoch[0])

In [8]:
# Start with a base simulation of the Sun, Planets and Moon
sim = make_sim_planets(epoch=epoch0)
# Set simulation time t to match epoch
sim.t = sim.epoch
# Set the number of active particles to the base simulation
# https://rebound.readthedocs.io/en/latest/ipython/Testparticles.html
sim.N_active = sim.N

Added bodies to simulation.


In [9]:
# Get the elements
# elts = sp2df(sp_name='KS.GetAsteroidRefElements', params={'epoch':epoch0})
elts = get_ast_ref_elts(epoch0)

In [10]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,f,M
0,1413593,60416640,2060 T-2,41956.0,2.373136,0.202053,0.012784,3.08049,3.456215,6.158491,6.202047


In [11]:
# First set of asteroid IDs
asteroid_ids = add_asteroid_elts(sim=sim, elts=elts)

In [12]:
asteroid_ids

array([1413593])

In [13]:
sim, elts = process_dates(epoch_out, max_dates=3)

In [14]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,M,f
0,1413593,84960000,2060 T-2,59000.0,2.319947,0.193103,0.013013,3.085343,-2.811390,-0.435800,5.640864
1,1413594,84960000,2678 T-3,59000.0,2.243868,0.265620,0.169600,-2.723455,2.611393,-1.980524,-2.398462
2,1000007,84960000,1979 MW5,59000.0,3.185015,0.298391,0.164023,2.169598,2.773325,1.322903,-4.361898


In [15]:
max_date = 3
N_date = 1000

In [19]:
schema='KS'
table='AsteroidElement_Ref'
truncate=False
chunksize=20**20
verbose=True
df2db(df=elts, schema=schema, table=table, truncate=truncate, chunksize=chunksize, verbose=verbose)

KeyError: "['period', 'omega_peri', 'eccentric_anomaly', 'Omega_node', 'mean_motion'] not in index"